In [ ]:
import requests
import warnings
import math

import pandas as pd
import datetime as dt
from datetime import date

import plotly.express as px

from tqdm import tqdm
from gql import gql, Client # to use GraphQL
from gql.transport.requests import RequestsHTTPTransport

%matplotlib inline

warnings.filterwarnings('ignore')

# Uniswap V3 position with hedge

Я выбрал изначальный пул, который был рассмотрен на семинаре, потому что одной из монет является стейблкоин, привязанный к доллару, а вторая менее волатильна по сравнению с другими монетами. Я не выбрал пул с двумя стейблкоинами, так как он приносит меньший доход, хотя и более стабилен. Остальные пулы имели меньшую ликвидность, что делало их поведение менее предсказуемым. (Выводы не убрал, потому что некоторые из них считаются больше 10 минут.)

## Заполнение датафрэйма

USDC - WETH pool: 0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640

In [ ]:
# connect to the Uniswap V3 Subgraph
uni_transport = RequestsHTTPTransport(
    url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3',
    verify = True,
    retries=3,
)
client = Client(transport = uni_transport)

In [ ]:
data_pool = []

# query for the USDC-WETH pool
query = gql(
    """
    {
        poolDayDatas(
            first: 1000
            orderBy: date
            orderDirection: desc
            where: {pool: "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"}
        ) {
            date
            token0Price
            token1Price
            liquidity
            volumeUSD
        }
    }
    """
)


result = client.execute(query, variable_values={"first": 1000, "skip": 0})

data = []
for i in result['poolDayDatas']:
    data.append([
        i['date'],
        i['token0Price'],
        i['token1Price'],
        i['liquidity'],
        i['volumeUSD']
    ])

df = pd.DataFrame(data)
df.columns = ['date', 'token0Price', 'token1Price', 'liquidity', 'volumeUSD']

df['token0Price'] = pd.to_numeric(df['token0Price'])
df['token1Price'] = pd.to_numeric(df['token1Price'])
df['liquidity'] = [float(num) for num in df['liquidity']]
df['USDC/WETH'] = pd.to_numeric(df['token0Price']/df['token1Price'])
df['datetime'] = pd.to_datetime(df['date']*1000000000)
df['volumeUSD'] = pd.to_numeric(df['volumeUSD'])

data_pool = df.copy()
data_pool['datetime'] = data_pool['datetime'].dt.date
data_pool = data_pool[['token0Price', 'token1Price', 'liquidity', 'datetime', 'volumeUSD']]

In [ ]:
# funding rates for ETH
import time

url = 'https://fapi.binance.com/fapi/v1/fundingRate'
symbol = 'ETHUSDT'

start_time = dt.datetime(2021, 5, 5).timestamp() * 1000
end_time = (time.time()-86400)* 1000

data = requests.get(url, params={'symbol': symbol, 'startTime': int(start_time)}).json()
last_time = data[-1]['fundingTime']


while last_time <= end_time:
    data.extend(requests.get(url, params={'symbol': symbol, 'startTime': int(last_time)}).json())
    last_time = data[-1]['fundingTime']

df = pd.DataFrame(data)
df['datetime'] = pd.to_datetime(df['fundingTime'], unit='ms')
df['fundingRate'] = pd.to_numeric(df['fundingRate'])
df = df.drop_duplicates()
data_funding = df.copy()
data_funding['datetime'] = data_funding['datetime'].dt.date
data_funding = data_funding.groupby('datetime').sum()
data_funding = data_funding.reset_index()
data_funding = data_funding[['datetime', 'fundingRate']]

In [ ]:
all_data = pd.merge(data_pool, data_funding, how="left", on='datetime')
all_data = all_data.sort_values('datetime').reset_index().drop('index', axis=1)

## Стратегия и backtest на исторических данных

In [ ]:
class UniswapV3Position:

    def __init__(self, token0_amount: float, token1_amount: float, liquidity: float,
                 price_current: float, price_upper: float, price_lower: float):
        self._token0_amount: float = token0_amount
        self._token1_amount: float = token1_amount
        self._liquidity: float = liquidity
        self._price_current: float = price_current
        self._price_init: float = price_current
        self._price_upper: float = price_upper
        self._price_lower: float = price_lower
        self._acc_fees: float = 0

    def update_state(self, price: float) -> None:
        if price < self._price_lower:
            self._token0_amount = 0
            self._token1_amount = self._liquidity * (1/(self._price_lower**0.5) - 1/(self._price_upper**0.5))
        elif self._price_lower <= price < self._price_upper:
            self._token0_amount = self._liquidity * (price**0.5 - self._price_lower**0.5)
            self._token1_amount = self._liquidity * (1/(price**0.5) - 1/(self._price_upper**0.5))
        else:
            self._token0_amount = self._liquidity * (self._price_upper**0.5 - self._price_lower**0.5)
            self._token1_amount = 0
        self._price_current = price


    def balance(self, side: bool = True) -> float:
        if side:
            return self._token0_amount + self._token1_amount * self._price_current + self._acc_fees
        return self._token0_amount / self._price_current + self._token1_amount + self._acc_fees / self._price_current

    @classmethod
    def price_to_tick(cls, price: float) -> float:
        return math.floor(math.log(price, 1.0001))

    @classmethod
    def tick_to_price(clas, tick: float) -> float:
        return 1.0001 ** tick

    def __str__(self) -> str:
        return f"token0: {self._token0_amount}, token1: {self._token1_amount}, liquidity: {self._liquidity}, price_current: {self._price_current}, price_upper: {self._price_upper}, price_lower: {self._price_lower}"

    def __repr__(self) -> str:
        return self.__str__()


def create_position_by_notional(
            deposit_amount_in_notional: float,
            price_current: float, price_upper: float, price_lower: float,
    ) -> UniswapV3Position:
        X = deposit_amount_in_notional

        liquidity = X / ((price_current**0.5 - price_lower**0.5)  + (1 / price_current**0.5 - 1 / price_upper**0.5) * price_current)
        token0_amount = liquidity * (price_current**0.5 - price_lower**0.5)
        token1_amount = liquidity * (1 / price_current**0.5 - 1 / price_upper**0.5)

        return UniswapV3Position(token0_amount, token1_amount, liquidity, price_current, price_upper, price_lower)

Решил выбрать дельта хэджирование. Реализовал его с помощью формул, найденных по ссылке, которую скинули в беседе.

In [ ]:
class HedgePosition:
    def __init__(self, size: float, entry_price: float):
        self._size: float = size
        self._entry_price: float = entry_price
        self._current_price: float = entry_price

    def update_state(self, price: float) -> None:
        self._current_price = price

    def pnl(self) -> float:
        return self._size * (self._current_price - self._entry_price)

    def __str__(self) -> str:
        return f'BaseHedgePosition(size={self._size}, entry_price={self._entry_price}, current_price={self._current_price})'


class Hedge:
    def __init__(self):
        self._position: HedgePosition = None
        self._balance: float = 0
        self._current_price: float = None
        self._trading_fee: float = 0.0003 # 0.03% комиссия взята с официального сайта

    def deposit(self, amount: float):
        if amount <= 0:
            raise Exception(f'Cannot deposit non-positive amount {amount}')
        self._balance += amount

    def withdraw(self, amount: float):
        if amount > self._balance:
            raise Exception(f'Not enough balance to withdraw {amount}, available {self._balance}')
        self._balance -= amount

    @property
    def balance(self) -> float:
        return self._balance

    @property
    def position(self) -> HedgePosition:
        return self._position

    def margin_balance(self) -> float:
        if not self._position:
            return self._balance
        return self._balance + self._position.pnl()

    def update_state(self, price: float, funding: float) -> None:
        self._current_price = price
        if self._position:
            self._position.update_state(price)
            self._balance += funding * abs(self._position._size) * self._current_price
        self.__check_liquidation()

    def open_position(self, size: float, entry_price: float) -> None:
        if self._position:
            raise Exception(f'Cannot open position, already have one {self._position}')

        if size > 0:
            raise Exception(f'Cannot open short position {size}')

        self._position = HedgePosition(size, entry_price)
        self._balance -= abs(size) * entry_price * self._trading_fee

    def close_position(self) -> None:
        if not self._position:
            return
        self._balance -= abs(self._position._size) * self._current_price * self._trading_fee
        self._balance += self._position.pnl()
        self._position = None

    def __check_liquidation(self):
        if self._position:
            # liquidate if margin balance is 5% of the position size
            if self.margin_balance() < abs(self._position._size) * 0.05:
                self._balance = 0
                self._position = None

In [ ]:
class Strategy:

    def __init__(self,
                 states: pd.DataFrame,
                ):
        self._hedge: Hedge = Hedge()
        self._pool_position: UniswapV3Position = None
        self._states: pd.DataFrame = states
        self._data = []

    def calculate_std(self, timestamp: dt.datetime) -> float:
        return abs(self._states[self._states['datetime'] <= timestamp]['token0Price'].pct_change().std())

    def calc_fees(self, price: float, i: dict, liquidity: float): 
        liquidityAmount1 = self._pool_position._token1_amount * (self._pool_position._price_upper**(0.5) * self._pool_position._price_lower**(0.5)) / (self._pool_position._price_upper**(0.5) - self._pool_position._price_lower**(0.5)) 
        liquidityAmount0 = self._pool_position._token0_amount / (self._pool_position._price_upper**(0.5) - self._pool_position._price_lower**(0.5)) 
 
        if i < 7:
            volume24H = self._states.loc[0:i]['volumeUSD'].mean()
        else:
            volume24H = self._states.loc[(i-7):i]['volumeUSD'].mean()

        if price < self._pool_position._price_lower: 
            deltaL = liquidityAmount1
        elif self._pool_position._price_lower <= price < self._pool_position._price_upper: 
            deltaL = min(liquidityAmount0, liquidityAmount1) 
        else: 
            deltaL = liquidityAmount0

        self._pool_position._acc_fees += 0.0005 * volume24H * (deltaL/ (liquidity*10**(-12) + deltaL)) # умножаю на 10^(-12), так как L=sqrt(xy) decimals_y=18 decimals_x=6 (-18-6)/2
        #print(self._pool_position._token0_amount, type(self._pool_position._token0_amount))
        #print(liquidityAmount0, liquidityAmount1, deltaL, liquidity*10**(-12),volume24H)
        #print(price, self._pool_position.balance(),self._pool_position._price_upper, self._pool_position._price_lower)
        #print(0.0005 * volume24H *  (deltaL / (liquidity*10**(-12) + deltaL)))

    def run(self, STD_COUNT: int, START_EQUITY: int):
        for i, state in tqdm(self._states.iterrows()):
            if i < 45: #рсновной объём появился в середине июня
                continue

            self._hedge.update_state(state['token0Price'], state['fundingRate'])

            if self._pool_position:
                self._pool_position.update_state(state['token0Price'])
                self.calc_fees(self._pool_position._price_current, i, state['liquidity'])
                if self._pool_position._token0_amount * self._pool_position._token1_amount == 0:
                    self.rebalance(state, START_EQUITY, STD_COUNT)
            else:
                self.rebalance(state, START_EQUITY, STD_COUNT)
            

            self._data.append([
                state['datetime'],
                state['token0Price'],
                self._pool_position.balance(),
                self._hedge.margin_balance(),
                self._pool_position.balance() + self._hedge.margin_balance(),
                self._pool_position._price_upper,
                self._pool_position._price_lower,
                self._pool_position._acc_fees
            ])

        return pd.DataFrame(self._data, columns=['datetime', 'price', 'pool_balance', 'hedge_balance', 'total_balance', 'price_upper', 'price_lower', 'fees'])
        
    def rebalance(self, state, START_EQUITY, STD_COUNT):

        std = self.calculate_std(state['datetime'])

        if self._pool_position is not None:
            pool_balance: float = self._pool_position.balance() * (1-0.005)  # опять же взял 0.001 с binance + 0.003 проскальзывание
            self._hedge.close_position()
            equity: float = pool_balance + self._hedge.margin_balance()
            HEDGE_RATIO = ((self._pool_position._price_upper/self._pool_position._price_lower)**0.25-1)/((self._pool_position._price_upper/self._pool_position._price_lower)**0.5-1)
            if HEDGE_RATIO <= 0:
                HEDGE_RATIO = 1/3
        else:
            equity: float = START_EQUITY
            HEDGE_RATIO = 1/3

        self._hedge.withdraw(self._hedge.margin_balance())
        self._hedge.deposit(HEDGE_RATIO * equity)
        

        self._pool_position = create_position_by_notional(
            (1-HEDGE_RATIO)*equity,
            price_current=state['token0Price'],
            price_upper=state['token0Price'] * (1 + std * STD_COUNT),
            price_lower=state['token0Price'] * (1 - std * STD_COUNT),
        )
        self._hedge.open_position(
            size=-(self._pool_position.balance() * HEDGE_RATIO /  (1-HEDGE_RATIO)) / state['token0Price'],
            entry_price=state['token0Price'],
        )


Пример с std_count = 3

In [ ]:
strategy = Strategy(all_data.sort_values('datetime').reset_index())
data = strategy.run(3, 100000)
df = pd.DataFrame(data)
px.line(df.sort_values('datetime'), x='datetime', y=['pool_balance', 'hedge_balance', 'total_balance'])

In [ ]:
px.line(df, x='datetime', y=['price', 'price_upper', 'price_lower'])

Найдём при каком значении достигается максимум.

In [ ]:
import numpy as np
import plotly.express as px

std_counts = np.arange(0.1, 10, 0.1)

parameters = []
final_total_balances = []

for std_count in std_counts:
    strategy = Strategy(all_data)
    df = pd.DataFrame(strategy.run(std_count, 100000))
    df['total_balance'] = df['total_balance'] / df['total_balance'].iloc[0]
        
    parameters.append(std_count)
    final_total_balance = df['total_balance'].iloc[-1]
    final_total_balances.append(final_total_balance)

data = {'std_count': std_counts, 'total_balance': final_total_balances}
df = pd.DataFrame(data)

fig = px.line(df, x='std_count', y='total_balance')
fig.update_traces(marker=dict(size=5))
fig.update_layout(scene=dict(xaxis_title='STD_COUNT', yaxis_title='Total Balance at End'))

fig.show()

In [ ]:
import numpy as np
import plotly.express as px

std_counts = np.arange(0.001, 0.15, 0.001)

parameters = []
final_total_balances = []

for std_count in std_counts:
    strategy = Strategy(all_data)
    df = pd.DataFrame(strategy.run(std_count, 100000))
    df['total_balance'] = df['total_balance'] / df['total_balance'].iloc[0]
        
    parameters.append(std_count)
    final_total_balance = df['total_balance'].iloc[-1]
    final_total_balances.append(final_total_balance)

data = {'std_count': std_counts, 'total_balance': final_total_balances}
df = pd.DataFrame(data)

fig = px.line(df, x='std_count', y='total_balance')
fig.update_traces(marker=dict(size=5))
fig.update_layout(scene=dict(xaxis_title='STD_COUNT', yaxis_title='Total Balance at End'))

fig.show()

Видно, что чем меньше STD_CONT, тем выше доходность.

## Метод Монте-Карло с генерацией цены эфириума и сохранением остальных парметров пула.

In [ ]:
eth_price_usd = all_data['token0Price']

price_changes = eth_price_usd.pct_change().dropna()

mu = price_changes.mean()
sigma = price_changes.std()

print(f"Среднее ежедневное изменение (mu): {mu}")
print(f"Стандартное отклонение (sigma): {sigma}")

Заполнение базы данных. Цена генерируется с помощью геометрического броунского движения.

In [ ]:
import numpy as np
import pandas as pd
import math

dataframes = []
num_dataframes = 10000

days = len(all_data)

for _ in range(num_dataframes):
    S = np.zeros(days + 1)
    S[0] = all_data['token0Price'][0]
    df = all_data.copy()

    for t in range(1, days):
        log = np.random.normal(mu - sigma**2 / 2, sigma)
        S[t] = math.exp(log) * S[t - 1]
        df['token0Price'][t] = S[t]

    dataframes.append(df)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.title('Визуализация траекторий token0Price')

for df in dataframes:
    plt.plot(df['datetime'], df['token0Price'], linewidth=0.5)

plt.xlabel('Дата')
plt.ylabel('token0Price')
plt.grid(True)
plt.show()

Реализация самого метода

In [ ]:
import numpy as np
import plotly.express as px

std_counts = [0.1,0.2,0.25,0.5,1,2,3,4,6,8,10] # Выбраны так, потому что из теста на исторических данных видна обратно пропорциональная зависимость результата и std_count

parameters = []
final_total_balances = []
max_total_balances = []
corresponding_parameters = []

for d in dataframes:
    for std_count in std_counts:
        strategy = Strategy(d)
        df = pd.DataFrame(strategy.run(std_count, 100000))
        df['total_balance'] = df['total_balance'] / df['total_balance'].iloc[0]
        
        parameters.append((std_count))
        final_total_balance = df['total_balance'].iloc[-1]
        final_total_balances.append(final_total_balance)
    max_total_balance = max(final_total_balances)
    max_total_balances.append(max_total_balance)
    max_total_balance_index = final_total_balances.index(max_total_balance)
    corresponding_parameter = parameters[max_total_balance_index] 
    corresponding_parameters.append(corresponding_parameter)
    
mean_max_total_balance = np.mean(max_total_balances)
mean_corresponding_parameter = np.mean(corresponding_parameters)
print(f"Mean of Maximum Total Balance: {mean_max_total_balance}")
print(f"Mean of corresponding Parameters (std_count): {mean_corresponding_parameter}")

Этот вывод подтверждает ранее выдвинутую гипотезу.

## Сценарное моделирование

### На росте

In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

dataframes_up = []
num_dataframes = 100

days = len(all_data)

for _ in range(num_dataframes):
    S = np.zeros(days + 1)
    S[0] = all_data['token0Price'][0]
    df = all_data.copy()

    for t in range(1, days):
        log = np.random.normal(2*mu - sigma**2 / 2, sigma) # увеличили мат ожидание, оно больше 0. А щначит тренд восхлдящий
        S[t] = math.exp(log) * S[t - 1]
        df['token0Price'][t] = S[t]

    dataframes_up.append(df)

plt.figure(figsize=(12, 6))
plt.title('Визуализация траекторий token0Price с восходящим трендом')

for df in dataframes_up:
    plt.plot(df['datetime'], df['token0Price'], linewidth=0.5)

plt.xlabel('Дата')
plt.ylabel('token0Price')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import plotly.express as px

final_total_balances = []

strategy = Strategy(all_data)

for d in dataframes_up:
    strategy = Strategy(d)
    df = pd.DataFrame(strategy.run(0.1, 100000))
    df['total_balance'] = df['total_balance'] / df['total_balance'].iloc[0]
        
    final_total_balance = df['total_balance'].iloc[-1]
    final_total_balances.append(final_total_balance)

mean_total_balance = np.mean(final_total_balances)

print(f"Mean Total Balance: {mean_total_balance}")

### Flat

In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

dataframes_flat = []
num_dataframes = 100

days = len(all_data)

for _ in range(num_dataframes):
    S = np.zeros(days + 1)
    S[0] = all_data['token0Price'][0]
    df = all_data.copy()

    for t in range(1, days):
        log = np.random.normal(0, sigma) 
        S[t] = math.exp(log) * S[t - 1]
        df['token0Price'][t] = S[t]

    dataframes_flat.append(df)

plt.figure(figsize=(12, 6))
plt.title('Визуализация траекторий token0Price flat')

for df in dataframes_flat:
    plt.plot(df['datetime'], df['token0Price'], linewidth=0.5)

plt.xlabel('Дата')
plt.ylabel('token0Price')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import plotly.express as px

final_total_balances = []


for d in dataframes_flat:
    strategy = Strategy(d)

    df = pd.DataFrame(strategy.run(0.1, 100000))
    df['total_balance'] = df['total_balance'] / df['total_balance'].iloc[0]
        
    final_total_balance = df['total_balance'].iloc[-1]
    final_total_balances.append(final_total_balance)

mean_total_balance = np.mean(final_total_balances)

print(f"Mean Total Balance: {mean_total_balance}")

### На падении

In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

dataframes_down = []
num_dataframes = 100

days = len(all_data)

for _ in range(num_dataframes):
    S = np.zeros(days + 1)
    S[0] = all_data['token0Price'][0]
    df = all_data.copy()

    for t in range(1, days):
        log = np.random.normal(mu / 2 - sigma**2 / 2, sigma) 
        S[t] = math.exp(log) * S[t - 1]
        df['token0Price'][t] = S[t]

    dataframes_down.append(df)

plt.figure(figsize=(12, 6))
plt.title('Визуализация траекторий token0Price flat')

for df in dataframes_down:
    plt.plot(df['datetime'], df['token0Price'], linewidth=0.5)

plt.xlabel('Дата')
plt.ylabel('token0Price')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import plotly.express as px

parameters = []
final_total_balances = []


for d in dataframes_down:
    strategy = Strategy(d)

    df = pd.DataFrame(strategy.run(0.1, 100000))
    df['total_balance'] = df['total_balance'] / df['total_balance'].iloc[0]
        
    final_total_balance = df['total_balance'].iloc[-1]
    final_total_balances.append(final_total_balance)

mean_total_balance = np.mean(final_total_balances)

print(f"Mean Total Balance: {mean_total_balance}")

Стратегия лучше показывает себя на падении. Это связано с тем, что существенную часть бюджет составляет хэджирование, а оно сделано лучше, чем сама стратегия.

# Вывод

Проведя исследование и разработав стратегию активного предоставления ликвидности в пуле на Uniswap V3 с дельта-нейтральным хеджированием позиции, я пришел к следующим выводам:

1. **Реалистичность стратегии**: По результатам бектестов и симуляций, стратегия, разработанная мной, хоть и показала высокую доходность, но оказалась не применимой в реальности, так как она основывается на ежедневном перемещении диапазона. Дни в данной модели - это точки. И в какой момент времени перемещать диапазон не понятно. А из-за того, что он маленький, цена за день с большой вероятностью выйдет за пределы установленного диапазона и комиссий мы не получим.

2. **Необходимость в симуляциях Монте-Карло**: Важным шагом оказалась симуляция Монте-Карло для оценки стратегии на разных траекториях цен. Это позволило оценить, как стратегия работает в разных сценариях, что особенно важно на волатильных криптовалютных рынках. Благодаря данному методу можно придти к мысли, что хэджирование сделанно верно.

3. **Уроки и дальнейшие шаги**: Из этого опыта я извлек важные уроки о сложности предоставления ликвидности на криптовалютных биржах. Для улучшения стратегии необходимо более глубокое исследование рынка, разработка более сложных моделей и учет большего количества факторов.

В заключение, хочу подчеркнуть, что разработка стратегии активного предоставления ликвидности на Uniswap V3 с хеджированием позиции – это сложная и многогранная задача. Несмотря на то, что моя стратегия не оказалась успешной, я смог больше понять Uniswap V3, а также узнать, как работать с API, датафреймами и классами.